# Diffusion Models

"Abraham Lincoln underwater playing the accordion, impressionist painting in the style of Monet" rendered by Stable Diffusion

<img src="https://i.imgur.com/2DattEn.png" width=400><br/>

*PyData PDX - October 12, 2022*

* Background
  * Sampling problem
  * Manifold hypothesis
  * Learned embeddings; latent spaces
* From GANs to recent diffusers
  * The foundations of modern text-to-image: VQGAN, CLIP
  * From CLIP+VQGAN to Stable Diffusion
  * Energy-based models, score-based models
  * General structure of recent diffusion models
* Building and sampling from a toy diffuser model
* Interpreting the Stable Diffusion paper diagram
* What we don’t understand: recent surprises


## Background

Today, we're not surprised or unreasonably impressed with synthetic images like these:

<img src="https://materials.s3.amazonaws.com/i/gan.jpg"><br/>

And we have access to even larger, better quality images via sites like
* https://thispersondoesnotexist.com/
* https://thiscatdoesnotexist.com/

These creations are the result of the GAN deep-learning pattern for image synthesis.

## Image generation as sampling

Image generation can be seen as a form of sampling from the distribution of "legitimate" images or conditional distribution of legitimate images corresponding to a prompt, class, etc.

### Key challenges and concepts

* Arbitrary and high-dimensional nature of pixel space
* Manifold hypothesis
* Embeddings
  * What are they?
  * Where do they get semantics from?
  * How/why use them?

### A few sampling mechanisms

* Rejection sampling
* Closed-form or simple approximation to transform samples from an easy distribution to a less-easy one
    * e.g., uniform to Gaussian 
    * via approximate inverse-CDF
* Variational methods (approximate the true distribution with one that's easier to manipulate and sample from)
* Markov chain Monte Carlo (MCMC) methods like Gibbs, Metropolis-Hastings, Hamiltonian, etc.

### Where do GANs fit in?

GANs
* are related to the second option above ... but on "ultra-hard mode" (neither a meaningful closed form nor simple approximation)
* use neural networks and a particular training algorithm to fit the transformation function
* yield interesting and useful compressed representations in embedding or latent space
* once trained, require just a single forward pass to move from a simple distribution (typically a high-dimensional Gaussian) to an output (usually an image) -- no MCMC
* Characteristic which defines GANs:
  * The "A" in GAN -- i.e., the adversarial training algorithm
  * Adversarial algorithm is also a vulnerability which can make training difficult

## From GANs to modern diffusers

<img src='https://materials.s3.amazonaws.com/i/astro.png' width=500 />

Perhaps the MNIST of 2022, the astronaut riding a horse has become emblematic of modern text-prompted diffusion models. We won't build a model of this quality here (for obvious reasons) but if you want to make your own equestrian spacemen or see a few pre-made specimens, it doesn't get easier than this notebook using Huggingface wrappers for accessing Stable Diffusion: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb

__Catching up__

Compressing an amazing amount of brilliant work into a tl;dr, the last couple of years look like this...

* VAE (prior work)
* VQVAE: VAE + quantized vectors (categorical/discrete)  
  * https://arxiv.org/pdf/1711.00937.pdf
  * Extensive explanation at https://ml.berkeley.edu/blog/posts/vq-vae/
  * DALL-E ~ Transformer (autoregressive generation) + VQVAE
* VQGAN ~ VQVAE discrete codebook idea + CNN + Transformers (on the image side!) + patch sequences + dominant perceptual loss
  * https://arxiv.org/pdf/2012.09841.pdf
* CLIP ~ image captioning by jointly training a text encoder and image encoder (https://arxiv.org/pdf/2103.00020.pdf)

__CLIP + VQGAN__

Pre-trained CLIP and pre-trained VQGAN

> We start with a text prompt and use a GAN to iteratively generate candidate images, at each step using CLIP to improve the image. We optimize the image by treating the squared spherical distance between the embedding of the candidate and the embedding of the text prompt as a loss function, and differentiating through CLIP with respect to the GAN’s latent vector representation
of the image (https://arxiv.org/abs/2204.08583)



__... getting to Stable Diffusion__

* Dall-e-2
  * CLIP-based with either (autoregressive or a flavor of __latent diffusion__) + (diffusion upsample) to generate encoded image vector (which is then expanded)
  * https://cdn.openai.com/papers/dall-e-2.pdf
  * You may have played with Dall-e-mini [a.k.a Craiyon - https://www.craiyon.com/] or at least seen it on social media (https://github.com/borisdayma/dalle-mini)
    * Here is a small, more accessible port: https://github.com/kuprel/min-dalle
* Parti
  * Based on improved VQGAN
  * https://parti.research.google/
  * https://ai.googleblog.com/2022/05/vector-quantized-image-modeling-with.html
* Imagen
  * Diffusion model with pre-trained LLM input
  * https://arxiv.org/abs/2205.11487
  * https://www.assemblyai.com/blog/how-imagen-actually-works/

Nice summary: https://twitter.com/savvyrl/status/1540555792331378688

We'll drill down on __diffusion models__ in a moment.

For context/situational awareness: once we have diffusion models, two more bits get us to Stable Diffusion, an open model based on Dall-e-2 concepts...

* Guided diffusion
  * More detail on class conditioning vs. classifier-free guidance for diffusion models (https://arxiv.org/abs/2207.12598)
* Latent guided diffusion (https://ommer-lab.com/research/latent-diffusion-models/)
  * Diffusion in the latent space instead of in the pixel space
  
We'll come back to closer look at Stable Diffusion but first...

### Code detail: diffusion model

If you've looked at any discussion of diffusion models, you've probably seen an image like this...

<img src='https://cdn-images-1.medium.com/max/820/1*RDPhd2dvmHE4UrAP-QHb9w.png' />

It's a cool idea ... but suggests a lot of questions:
* Why would we ever expect this sort of thing to work?
  * If you've seen denoising networks, you might instead say, "I buy the part on the left ... but how does the 'reverse' process start?"
* If it appears to work for some samples ... can we rely on it to cover the target distribution well?
* Is it efficient (computationally)?

Diffusion models are a form of energy-based model -- models inspired by physical knowledge (typically but not always from statistical mechanics) of how certain processes involving adjusting energy and entropy levels systematically can produce a known set of world states. See https://towardsdatascience.com/the-physics-of-energy-based-models-1121122d0d9

That's a mouthful... so let's try to explain a bit more.

One very classic example you may have seen or implemented is simulated annealing (https://en.wikipedia.org/wiki/Simulated_annealing)

Another case, a bit closer to deep learning, is the Hopfield network. The late David MacKay's coverage (from the epic *Information Theory, Inference, and Learning Algorithms*): http://www.inference.org.uk/mackay/itprnn/ps/504.520.pdf

In our case, where we are looking not for a single optimum, but a complex distribution, what we want is a
* training mechanism that places the target (training) distribution at low-energy places in the parameter-energy landscape
* sampling mechanism that can provably (or at least practically) yield low-energy configurations
* justification for believing the output configurations are distributed properly
  * in the simple case, this is avoiding mode collapse
  * in the broader case, this is accessing all parts of the training distribution with the relevant likelihoods
  
Key papers include
* Ho et al. -- https://arxiv.org/abs/2006.11239 -- from which we'll get the algorithm used below
* and Song et al. -- https://arxiv.org/abs/2011.13456 
  * which is recapped in the excellent blog "Generative Modeling by Estimating Gradients of the Data Distribution" (by Yang Song) at https://yang-song.net/blog/2021/score/
  
This blog post -- https://lilianweng.github.io/posts/2021-07-11-diffusion-models/ -- which provides more detail (and math) than we have time for today, is not just a great explainer on diffusion models -- its list of citations is a "greatest hits" of recent work linking diffusion, score-based models, Langevin dynamics, and efficient approximations of target processes. 

Essentially, we want not just a method, but also its justification, scope, and limitations.

__Context__

To set the scene, we want
* a training loop that learns a denoising-step function for our specific training set
* a sampling loop that uses the trained function together with noise injection to move toward the target distribution

Let's walk through a minimal implementation by Francois Fleuret. The full code is at https://fleuret.org/git-extract/pytorch/minidiffusion.py but here we've trimmed it down so we can highlight the critical bits.

In [ ]:
# Any copyright is dedicated to the Public Domain.
# https://creativecommons.org/publicdomain/zero/1.0/

# Written by Francois Fleuret <francois@fleuret.org>

import torch
import math, argparse
import matplotlib.pyplot as plt
import torchvision
from torch import nn
from torch.nn import functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

For training purposes, we need data samples. Here we'll generate them ourselves

In [ ]:
def sample_disc_grid(nb):
    a = torch.rand(nb) * math.pi * 2
    b = torch.rand(nb).sqrt()
    N = 4
    q = (torch.randint(N, (nb,)) - (N - 1) / 2) / ((N - 1) / 2)
    r = (torch.randint(N, (nb,)) - (N - 1) / 2) / ((N - 1) / 2)
    b = b * 0.1
    result = torch.empty(nb, 2)
    result[:, 0] = a.cos() * b + q
    result[:, 1] = a.sin() * b + r
    return result

def sample_spiral(nb):
    u = torch.rand(nb)
    rho = u * 0.65 + 0.25 + torch.rand(nb) * 0.15
    theta = u * math.pi * 3
    result = torch.empty(nb, 2)
    result[:, 0] = theta.cos() * rho
    result[:, 1] = theta.sin() * rho
    return result

Gather a minimum set of configs/hyperparams

In [ ]:
import sys

sys.argv = ['']

parser = argparse.ArgumentParser(
    description = '''A minimal implementation of Jonathan Ho, Ajay Jain, Pieter Abbeel
      "Denoising Diffusion Probabilistic Models" (2020) https://arxiv.org/abs/2006.11239''',
    formatter_class = argparse.ArgumentDefaultsHelpFormatter
)

parser.add_argument('--seed', type = int, default = 0, help = 'Random seed, < 0 is no seeding')
parser.add_argument('--nb_epochs', type = int, default = 40, help = 'How many epochs')
parser.add_argument('--batch_size', type = int, default = 25, help = 'Batch size')
parser.add_argument('--nb_samples', type = int, default = 25000, help = 'Number of training examples')
parser.add_argument('--learning_rate', type = float, default = 1e-3, help = 'Learning rate')

args = parser.parse_args()

if args.seed >= 0:
    torch.manual_seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

Now we'll get a set of training samples.

We'll also capture the means and stds of the inputs, which will be critical later.

In [ ]:
# train_input = sample_disc_grid(args.nb_samples).to(device)
train_input = sample_spiral(args.nb_samples).to(device)

train_mean, train_std = train_input.mean(), train_input.std()

The neural net we'll use here is intentionally simple. The only unusual piece is the `TimeAppender` which adds a new dimension to capture the time step at which we're operating ... the time step itself is an important parameter. I.e., the network is allowed to know where it is in the diffusion process, a key bit of "special sauce" for making it work.

We'll define the `TimeAppender` then the network which uses it.

In [ ]:
# Gets a pair (x, t) and appends t (scalar or 1d tensor) to x as an additional dimension / channel

class TimeAppender(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, u):
        x, t = u
        if not torch.is_tensor(t):
            t = x.new_full((x.size(0),), t)
        t = t.view((-1,) + (1,) * (x.dim() - 1)).expand_as(x[:,:1])
        return torch.cat((x, t), 1)

What would it look like to pass some samples and a time step through this module?

In [ ]:
demo = sample_spiral(5)
demo

In [ ]:
demo_ta = TimeAppender()
t_0 = 17

demo_ta((demo, t_0))

In [ ]:
nh = 256

model = nn.Sequential(
    TimeAppender(),
    nn.Linear(train_input.size(1) + 1, nh),
    nn.ReLU(),
    nn.Linear(nh, nh),
    nn.ReLU(),
    nn.Linear(nh, nh),
    nn.ReLU(),
    nn.Linear(nh, train_input.size(1)),
)

model.to(device)

The training loop will use a set of values for adjusting energy levels (noise mixing) based on a schedule
* with 1000 total steps ("T")
* linearly spaced from 1e-4 to 0.02
* a handful of derived quantities
  * including a set of precomputed cumulative products (log-sum-exp) that allow direct access to arbitrary steps in the noising process

In [ ]:
T = 1000
beta = torch.linspace(1e-4, 0.02, T)
alpha = 1 - beta
alpha_bar = alpha.log().cumsum(0).exp()
sigma = beta.sqrt()

plt.plot(beta, label='beta')
plt.plot(alpha, label='alpha')
plt.plot(alpha_bar, label='alpha_bar')
plt.plot(sigma, label='sigma')
plt.yscale('log')
plt.legend()

In [ ]:
demo

We're going to sample time steps and make a tensor with additional axes so that scalars from the energy schedule get broadcast across a tensor that matches the data dimensionality.

In [ ]:
t_demo = torch.randint(T, (demo.size(0),) + (1,) * (demo.dim()-1)) #1 below

t_demo

In [ ]:
torch.randint(T, (demo.size(0),) + (1,) * (demo.dim()-1)).shape

In [ ]:
demo_2 = torch.ones(5,2)

demo_2

In [ ]:
alpha_bar[17] * demo_2

In [ ]:
alpha_bar[t_demo] * demo_2 #2 below

Those tensors will be used for mixing -- part training sample and part Gaussian noise.

We'll then pass that mixture and an explicit scaled version of the timestep through the model ... and minimize the MSE of a noise prediction.

We're going to implement the training algorithm (and later the sampling algorithm) per Ho et al.:

<img src='https://materials.s3.amazonaws.com/i/ho-alg.png' width=700 />

In [ ]:
alpha_bar = alpha_bar.to(device)

for k in range(args.nb_epochs):

    acc_loss = 0
    optimizer = torch.optim.Adam(model.parameters(), lr = args.learning_rate)

    for x0 in train_input.split(args.batch_size):
        x0 = (x0 - train_mean) / train_std # standardize
        t = torch.randint(T, (x0.size(0),) + (1,) * (x0.dim() - 1), device = x0.device) #1 sample timesteps
        eps = torch.randn_like(x0).to(device) # sample Gaussian
        xt = torch.sqrt(alpha_bar[t]) * x0 + torch.sqrt(1 - alpha_bar[t]) * eps #2 mix
        output = model((xt, t / (T - 1) - 0.5))
        loss = (eps - output).pow(2).mean() # MSE to learn eps (noise)
        acc_loss += loss.item() * x0.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'{k} {acc_loss / train_input.size(0)}')

At this point we have a denoising model that should capture the training distribution.

If we were using a "real" model ... say, Stable Diffusion, this is the point we'd get to by downloading the pre-trained weight checkpoint. The distribution is "in" the model ... but we still need a pseudo-MCMC process to find samples.

We'll start with a Gaussian vector and repeatedly run it through our noise predictor,
* each time removing a scaled amount of the predicted noise
* adding new noise (per the sigma schedule)

Reminder: here, the time step ("T") counts down -- that's what makes this process go "backwards" even though the model always goes in one direction, from (image, timestep) -> noise estimation

In [ ]:
def generate(size, T, alpha, alpha_bar, sigma, model, train_mean, train_std):
    
    model.eval()
    with torch.no_grad():

        x = torch.randn(size, device = device)

        for t in range(T-1, -1, -1):
            output = model((x, t / (T - 1) - 0.5))
            z = torch.zeros_like(x) if t == 0 else torch.randn_like(x)
            x = 1/torch.sqrt(alpha[t]) \
                * (x - (1-alpha[t]) / torch.sqrt(1-alpha_bar[t]) * output) \
                + sigma[t] * z

        x = x * train_std + train_mean

        return x


x = generate((1200, 2), T, alpha, alpha_bar, sigma, model, train_mean, train_std)

Now we can look at our samples...

In [ ]:
fig = plt.figure()
fig.set_figheight(6)
fig.set_figwidth(6)
ax = fig.add_subplot(1, 1, 1)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)

d = train_input[:x.size(0)].detach().to('cpu').numpy()
ax.scatter(d[:, 0], d[:, 1], s = 2.5, color = 'gray', label = 'Train')

d = x.detach().to('cpu').numpy()
ax.scatter(d[:, 0], d[:, 1], s = 2.0, color = 'red', label = 'Synthesis')

ax.legend(frameon = False, loc = 2)

### Back to Stable Diffusion (and its discontents)

<img src='https://pbs.twimg.com/media/FasS_3UVsAAOAKh?format=png' width=700/>

As discussed, the key differences include...

* Diffusion in the latent space
* Conditioning/guiding via input prompt
* Attention layers to let the network learn how best to use the information in the prompt

The amazing results make this look like a solved problem ... the "ImageNet moment" of image generation, akin to the breakthroughs of CNNs and Transformers. And in many respects it is.

But there are some issues remaining, including
* runtime costs and optimality (https://twitter.com/yimatweets/status/1560744922415652864?s=21&t=cKqyctaFgSu96ZVpRxZ57g)
* the fact that other noising methods work when key parts of the key theory are not applied
  * https://twitter.com/tomgoldsteincs/status/1562503814422630406?s=21&t=paGhXB0cZohusg82uvIjXQ
  * https://arxiv.org/abs/2208.09392

The latest SOTA (as of September 2022) places these models within a superclass of "soft score-matching models" that may address part of these gaps: https://arxiv.org/abs/2208.09392

## Hyperspeed

- DALL•E 2 [6 Apr 2022]
- Imagen [23 May 2022]
- Stable Diffusion [22 Aug 2022]
- Make-A-Video [29 Sep 2022]
- Imagen-video [6 Oct 2022]

And there's more...
- Human Motion Diffusion Model https://guytevet.github.io/mdm-page/

Longer videos using an alternate approach: https://phenaki.github.io/


